In [3]:
%reset -f
%load_ext autoreload
%autoreload 2

import Odin as backend

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# validation test for scalar incompressible flow


geom = backend.createGeometry( 'rectangle', [1,1] )
mesh = backend.createMesh( geom, res=10 )

myFlowModels = {
    'flowField'     :   backend.FlowModels.IncompressibleFlow(),
    }

sim1 = backend.Simulation(myFlowModels, mesh, geom )

myFields = sim1.getFields()
U = myFields['U']
U.setBoundaryCondition('top', [1,0])
U.setBoundaryCondition('bottom', [0,0])
U.setBoundaryCondition('left', [0,0])
U.setBoundaryCondition('right', [0,0])

sim1.showfields()
sim1.updateLinSystems()
#U.solve()

variable fields :
	 U 	 variableFaceField 	 {'left': [0, 0], 'right': [0, 0], 'top': [1, 0], 'bottom': [0, 0]}
	 p 	 variableCellField 	 {'left': None, 'right': None, 'top': None, 'bottom': None}
parameter fields :
	 nu 	 scalar




AttributeError: 'IncompressibleFlow' object has no attribute 'updateLinearEquationSystems'

In [ ]:
sim1.display(U)
